In [16]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor

import warnings


In [14]:
df=pd.read_csv('/Users/aparnamallik/Documents/MLproject/notebook/data/stud.csv')
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [19]:
x=df.drop('math_score',axis=1)

In [20]:
x.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [21]:
y=df['math_score']

In [24]:
#seprating cat and num values
num_features=[]
cat_features=[]

for i in x.columns:
    if x[i].dtype=='object':
        cat_features.append(i)

for i in x.columns:
    if x[i].dtype!='object':
        num_features.append(i)

num_features

['reading_score', 'writing_score']

In [25]:
#transformation of num and cat
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor=ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),
    ]
)


In [27]:
X = preprocessor.fit_transform(x)

In [28]:
X.shape

(1000, 19)

In [33]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [34]:
x_train.shape, x_test.shape

((800, 19), (200, 19))

In [47]:
#evaluation function

def evalution(true,predicted):
    mse=mean_squared_error(true,predicted)
    mae=mean_absolute_error(true,predicted)
    r2_square=r2_score(true,predicted)
    rmse=np.sqrt(mse)
    return mae,r2_square,rmse
    

In [61]:
#model training
models={"LinearRegression": LinearRegression(), 
            "Ridge":Ridge(), 
            "Lasso":Lasso(),
            "KNN":KNeighborsRegressor(),
            "DT":DecisionTreeRegressor(),
            "RF":RandomForestRegressor(),
            "AdaBoost Regressor": AdaBoostRegressor(),
            "SVR":SVR(),
            "CatBoostRegressor":CatBoostRegressor(verbose=False)}
model_list=[]
r2_list=[]

for i in range(len(models)):

    model=list(models.values())[i]
    model.fit(x_train,y_train)

    y_train_pred=model.predict(x_train)
    y_test_pred=model.predict(x_test)

    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evalution(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evalution(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

LinearRegression
Model performance for Training set
- Root Mean Squared Error: 0.8743
- Mean Absolute Error: 4.2667
- R2 Score: 5.3231
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.8804
- Mean Absolute Error: 4.2148
- R2 Score: 5.3940


Ridge
Model performance for Training set
- Root Mean Squared Error: 0.8743
- Mean Absolute Error: 4.2650
- R2 Score: 5.3233
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.8806
- Mean Absolute Error: 4.2111
- R2 Score: 5.3904


Lasso
Model performance for Training set
- Root Mean Squared Error: 0.8071
- Mean Absolute Error: 5.2063
- R2 Score: 6.5938
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.8253
- Mean Absolute Error: 5.1579
- R2 Score: 6.5197


KNN
Model performance for Training set
- Root Mean Squared Error: 0.8550
- Mean Absolute Error: 4.5270
- R2 Score: 5.7172
----------------------------------
Model p

In [62]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
7,SVR,8.126623
4,DT,8.005935
3,KNN,7.255260
2,Lasso,6.519695
5,RF,6.041819
8,CatBoostRegressor,6.008632
6,AdaBoost Regressor,6.008470
0,LinearRegression,5.393994
1,Ridge,5.390387


In [64]:
pred_df=pd.DataFrame({'Actual Value':y_test,'Predicted Value':y_test_pred,'Difference':y_test-y_test_pred})
pred_df

,Actual Value,Predicted Value,Difference
521,91,74.709499,16.290501
737,53,54.706876,-1.706876
740,80,76.842097,3.157903
660,74,76.580288,-2.580288
411,84,87.297491,-3.297491
...,...,...,...
408,52,47.095579,4.904421
332,62,58.745708,3.254292
208,74,69.989700,4.010300
613,65,68.313052,-3.313052
